<h1>Using TFRecords to train ResNet50 in TensorFlow</h1>


<strong>This tutorial will explain how to train a "transfer learning" model using TRFrecords.</strong>

An interactive version of the notebook is available on Kaggle at:<br>
https://www.kaggle.com/spiroganas/using-tfrecords-to-train-resnet50-in-tensorflow
<hr>

Kaggle's [RANZCR CLiP - Catheter and Line Position Challenge](https://www.kaggle.com/c/ranzcr-clip-catheter-line-classification) provides data in the [TFRecord format](https://www.tensorflow.org/tutorials/load_data/tfrecord).  TensorFlow can quickly read data stored in the TFRecord format.  This is especially important when the model is being trained on fast GPUs or [TPUs](https://cloud.google.com/tpu) (where IO is frequently the training bottleneck).

This notebook shows how to to use TFRecords, TensorFlow and a GPU to train a ResNet50 model. 



In [ ]:
import tensorflow as tf
from io import BytesIO
from imageio import imread
from IPython import display
import cv2 as cv
import os
import random
import csv
import glob

In [ ]:
# Some constants
IMAGE_SIZE = 300

# Set this to True if you want to print arrays and other stuff that makes the notebook hard to read.
VERBOSE = False

In [ ]:
# View the TFRecord files provided by the "RANZCR CLiP - Catheter and Line Position Challenge"
!ls '/kaggle/input/ranzcr-clip-catheter-line-classification/test_tfrecords'


# Step 1: Load the TFRecord files into a tf.data.Dataset

https://www.tensorflow.org/guide/data

TFRecord files can be loaded into a tf.data.Dataset.
The dataset's map() method can then be used to preprocess the data.  https://www.tensorflow.org/api_docs/python/tf/data/Dataset#map




Datasets can  can be fed directly into your model's model.fit().  They can also be "optimized" to speed up the training process.

In [ ]:
INPUT_DATA_FOLDER = '/kaggle/input/ranzcr-clip-catheter-line-classification/train_tfrecords/'


# Get a list of the TFRecord files
TFRecordFiles = []
for dirname, _, filenames in os.walk(INPUT_DATA_FOLDER):
    for filename in filenames:
        if filename[-6:]=='.tfrec':
            TFRecordFiles.append(os.path.join(dirname, filename))

            
# Spilt the data into training and validation datasets                 
SplitNumber = int(.8*len(TFRecordFiles))  
train_dataset = tf.data.TFRecordDataset(TFRecordFiles[:SplitNumber])
val_dataset = tf.data.TFRecordDataset(TFRecordFiles[SplitNumber:])
        
    
    
# I looked this up in windows explorer
# It's the number of images in the train folder
DATASET_SIZE = 30083

train_size = int(0.8 * DATASET_SIZE)
val_size = int(0.2 * DATASET_SIZE)

# Create a training and a validation datasets
full_dataset = tf.data.TFRecordDataset(TFRecordFiles)
full_dataset = full_dataset.shuffle(buffer_size=1000)
train_dataset = full_dataset.take(train_size) #.cache()
val_dataset = full_dataset.skip(train_size).take(val_size) #.cache()
        
        

# Step 2: See what data is included in a single example

You need to create a "feature dictionary" that will be used to parse the TFRecord files into a format that can be used by your model.

To do that, you need to know what data is in a TFRecords, and the data's datatype.



In [ ]:
for raw_record in train_dataset.take(1):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    if False:  # Change this to True if you want to see the data.  I turned it off because it's long and it make this notebook hard to read.
        print(example)

# Step 3: Create a feature dictionary

The feature dictionary should describe the data stored in the TFRecord
For more details, see:  https://www.tensorflow.org/tutorials/load_data/tfrecord#read_the_tfrecord_file

In [ ]:
feature_dictionary = {
    'CVC - Abnormal': tf.io.FixedLenFeature([], tf.int64),
    'CVC - Borderline': tf.io.FixedLenFeature([], tf.int64),
    'CVC - Normal': tf.io.FixedLenFeature([], tf.int64),
    'ETT - Abnormal': tf.io.FixedLenFeature([], tf.int64),
    'ETT - Borderline': tf.io.FixedLenFeature([], tf.int64),
    'ETT - Normal': tf.io.FixedLenFeature([], tf.int64),
    'NGT - Abnormal': tf.io.FixedLenFeature([], tf.int64),
    'NGT - Borderline': tf.io.FixedLenFeature([], tf.int64),
    'NGT - Incompletely Imaged': tf.io.FixedLenFeature([], tf.int64),
    'NGT - Normal': tf.io.FixedLenFeature([], tf.int64),
    'StudyInstanceUID': tf.io.FixedLenFeature([], tf.string),    
    'Swan Ganz Catheter Present': tf.io.FixedLenFeature([], tf.int64),
    'image': tf.io.FixedLenFeature([], tf.string),
}

# Step 4: Parse the TFRecord

In this step, we parse the TFRecord data into a more useable format.

In [ ]:
# Define two parsing functions that will turn the TFRecord back into an array and a label        
def _parse_function(example, feature_dictionary=feature_dictionary):
    # Parse the input `tf.train.Example` proto using the feature_dictionary.
    # Create a description of the features.
    parsed_example = tf.io.parse_example(example, feature_dictionary)
    return parsed_example

train_dataset = train_dataset.map(_parse_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
val_dataset = val_dataset.map(_parse_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
print(train_dataset)


# Step 5:  Print a couple images

In [ ]:
for image_features in train_dataset.take(2):
    image = image_features['image'].numpy()
    display.display(display.Image(data=image))

# Step 6: Look at an image as an numpy ndarray

In [ ]:
if VERBOSE:
    for image_features in train_dataset.take(1):
        image = image_features['image'].numpy()
        numpyArray = imread(BytesIO(image))
        print(numpyArray)

# Step 7: Look at the labels

In [ ]:
for image_features in train_dataset.take(1):
    print('StudyInstanceUID: ', image_features['StudyInstanceUID'].numpy())
    print('CVC - Abnormal: ', image_features['CVC - Abnormal'].numpy())
    print('CVC - Borderline: ', image_features['CVC - Borderline'].numpy())
    print('CVC - Normal: ', image_features['CVC - Normal'].numpy())
    print('ETT - Abnormal: ', image_features['ETT - Abnormal'].numpy())
    print('ETT - Borderline: ', image_features['ETT - Borderline'].numpy())
    print('ETT - Normal: ', image_features['ETT - Normal'].numpy())
    print('NGT - Abnormal: ', image_features['NGT - Abnormal'].numpy())
    print('NGT - Borderline: ', image_features['NGT - Borderline'].numpy())
    print('NGT - Incompletely Imaged: ', image_features['NGT - Incompletely Imaged'].numpy())
    print('NGT - Normal: ', image_features['NGT - Normal'].numpy())
    print('Swan Ganz Catheter Present: ', image_features['Swan Ganz Catheter Present'].numpy())
    
    


# Step 8: Data Augmentation

Data Augmentation can improve the performance of you model. It applies transformations to the real images to generate "fake" (but realistic) images.  This increases the size of your training data, which often increases the accuaracy of your model.

This keras model can be used to add data augmentation to the TFDataset
https://www.tensorflow.org/tutorials/images/data_augmentation#two_options_to_use_the_preprocessing_layers

In [ ]:
# There are a lot of possible ways to transform an image.

data_augmentation = tf.keras.Sequential([
        tf.keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),    
    
        # Randomly pick about 83% of the area of the image (1/(1.1^2))                                       
        #tf.keras.layers.experimental.preprocessing.Resizing(height=int(1.1*IMAGE_SIZE), width=int(1.1*IMAGE_SIZE+100), interpolation='bilinear'),
        #tf.keras.layers.experimental.preprocessing.RandomCrop(height=IMAGE_SIZE, width=IMAGE_SIZE),

        # Changes the contrast.  Not required if using CT scan data that has been converted to Housfield Units.
        #tf.keras.layers.experimental.preprocessing.RandomContrast(factor=0.1 ),

        # Randomly flip the image left/right and/or up/down 
        tf.keras.layers.experimental.preprocessing.RandomFlip(),
    
        # Randomly rotate the image
        tf.keras.layers.experimental.preprocessing.RandomRotation(factor=(-0.2, 0.2), fill_mode='constant'),
], name="Data_Augmentation")




# Step 9: Generate (feature, label) pairs

The "features" are the data you are using to make your prediction (i.e. the medical image)
The "labels" are what you are trying to predict.

We want to transform the TFRecord feature (a string of bytes) back into an image.  Then we want to conver the image from greyscale to RGB color, and change it's size.
For our labels, we just want to turn them into one long list of zeros and ones.

In [ ]:
# Define two parsing functions that will turn the TFRecord back into an array and a label        
def generate_training_example(example):

    # Convert the image to an ndarray, resize it and convert it to RGB color
    # These are the settings most commonly required by base models used in transfer learning.

    features =  tf.io.decode_image(example['image'], expand_animations = False)
    features =  tf.image.grayscale_to_rgb(features)
    features =  tf.image.resize(features,size=(IMAGE_SIZE,IMAGE_SIZE))
    
    
    
    #features = example['image'].numpy()
    #features = imread(BytesIO(features))
    #features = cv.resize(features, new_image_size)
    #features = cv.cvtColor(features, cv.COLOR_GRAY2RGB)
    labels = [ # Edit this to add whatever labels you want your model to predict
                example['CVC - Abnormal'],
                example['CVC - Borderline'],
                example['CVC - Normal'],
                example['ETT - Abnormal'],
                example['ETT - Borderline'],
                example['ETT - Normal'],
                example['NGT - Abnormal'],
                example['NGT - Borderline'],
                example['NGT - Incompletely Imaged'],
                example['NGT - Normal'],
                example['Swan Ganz Catheter Present'],
            ]

    return features, labels


train_dataset = train_dataset.map(generate_training_example, num_parallel_calls=tf.data.experimental.AUTOTUNE) # .cache('/kaggle/temp/train.cache')
val_dataset = val_dataset.map(generate_training_example, num_parallel_calls=tf.data.experimental.AUTOTUNE) # .cache('/kaggle/temp/test.cache')





train_dataset = train_dataset.batch(32)
val_dataset = val_dataset.batch(32)

# Apply data augmentation to the training data set
train_dataset = train_dataset.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=tf.data.experimental.AUTOTUNE)




train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)
val_dataset = val_dataset.prefetch(tf.data.experimental.AUTOTUNE)


if VERBOSE:
    for X in train_dataset.take(1):
        print(X[0])
        print('-------------------')
        print(X[1])

# Step 10: Create a ResNet50 model

Nothing fancy, I just want to show that the above code works.

This is an example of "Transfer Learning".  We are starting with an existing, trained model.  We are adding a new "classification head" and then training the model to predict our labels.


In [ ]:
def create_model():
    
    n_labels = 11 # number or output classes
    
    auc = tf.keras.metrics.AUC(multi_label=True) # metric for multi-class multi-label models


    # https://keras.io/api/applications/
    # Options include: ResNet50, MobileNetV2
    base_model = tf.keras.applications.ResNet50(
        input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3), include_top=False, weights="imagenet"
    )

    base_model.trainable = True

    inputs = tf.keras.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
    
    # built-in resnet preprocessor
    #https://www.tensorflow.org/api_docs/python/tf/keras/applications/resnet/preprocess_input
    x = tf.keras.applications.resnet.preprocess_input(inputs)  
    
    x = base_model(x, training=False)
    

    # Convert features of shape `base_model.output_shape[1:]` to vectors
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    # A Dense classifier with a single unit (binary classification)
    outputs = tf.keras.layers.Dense(n_labels, activation='sigmoid')(x)
    model = tf.keras.Model(inputs, outputs)

    model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    #loss=custom_loss_fn,  # Custom Code to apply class weights
    metrics=[auc])


    return model




my_model = create_model()
print('New model created!')
print()
print(my_model.summary())

# Step 11: Train and Evaluate the ResNet50 model

We will use "callbacks"to produce a better model

The lr_reducer_callback callback will slowly reduce the learning rate as the model trains.

The EarlyStopping_callback will stop training the model once the model has achieved it's best fit.

In [ ]:
# Callback for decaying the learning rate.
lr_reducer_callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_auc", factor=0.1, patience=3, min_lr=1e-6, mode='max',verbose=1)

# Callback that stops training once the model has stopped imprtoving
EarlyStopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_auc', 
    min_delta=0.0001, 
    patience=5, 
    verbose=1,
    mode='max', 
    restore_best_weights=True
)





history = my_model.fit(
    train_dataset,
    epochs=100,
    validation_data=val_dataset,
    callbacks=[lr_reducer_callback,
                EarlyStopping_callback
                ],
)

# Save the trained model
my_model.save("/kaggle/working/ResNet50_Saved_Model.h5" , save_format="h5")


# test on the whole evaluiation dataset
results = my_model.evaluate(val_dataset)
print("Best Model Multi-label AUC: ", round(results[1],4) )
